# Python setup

In [1]:
import subprocess 
import time
import glob
import os
from pathlib import Path
import shutil
import pandas as pd
import tempfile
from PIL import Image
import resource

def run(obj,print_output=False):
    start = time.perf_counter()
    result = subprocess.run(obj["args"],capture_output=True,text=True)    
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        result.check_returncode()
    else:
        if print_output:
            print(result.stdout)
        return { "delta": time.perf_counter() - start, "label": obj["label"],"output": result.stdout,"error": result.stderr, "return_code": result.returncode }
    
rid="linux-x64"
#converter_exe_rel="../../bin/release/net6.0/linux-x64/GraphLogicA"
converter_exe_rel=f"../../bin/release/net6.0/{rid}/GraphLogicA"
converter_exe = Path(converter_exe_rel).absolute().as_posix()
graphlogica_exe = converter_exe
minimizer_exe = shutil.which("ltsconvert")
voxlogica_exe = f"../../VoxLogicA_1.0-experimental_{rid}/VoxLogicA"
output="output"
#shutil.rmtree(output,ignore_errors=True)
os.makedirs(output,exist_ok=True)
images = glob.glob("test-images/*.png")

def mk_df(results,delta_label):
    return pd.DataFrame(results).set_index("label").rename(columns={"delta": delta_label}).drop(columns=["output","error","return_code"])
#!(cd ../.. && dotnet build -c release -r $rid)
resource.setrlimit(resource.RLIMIT_STACK, (resource.RLIM_INFINITY, resource.RLIM_INFINITY))

# Convert images

In [2]:
def converter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix()], "label": label }
    #return (run(label,args))

converter_df = mk_df([ run(converter(image)) for image in images ],"conversionAndWrite")


# Minimize

In [3]:
def minimizer(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    return { "args": [minimizer_exe,"-ebranching-bisim",d_path.as_posix(),m_path.as_posix()], "label": label }

minimizer_df = mk_df([ run(minimizer(image)) for image in images ],"minimization")


# Convert without writing the file 

In [4]:
def fakeConverter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix(),"--fakeconversion"], "label": label }
    #return (run(label,args))

fakeConverter_df = mk_df([ run(fakeConverter(image)) for image in images ],"conversion")


# Model checking on images using VoxLogicA

In [5]:
def mazeSpecification(path):
    return f'''
    load img = "{path}"
    let colour(img,r,g,b) = (red(img) =. r) & (green(img) =. g) & (blue(img) =. b)
    let w = colour(img,255,255,255)
    let b = colour(img,0,0,255)
    let g = colour(img,0,255,0)
    let form1 = through(N b,w) & through(N g,w)
    let form2 = through(N (w & !through(N g,w)),b) & through(N g,w)
    let form3 = through(N form1,b)
    save "output/{path}/form1.png" form1
    save "output/{path}/form2.png" form2
    save "output/{path}/form3.png" form3
    '''

def modelChecker(image):
    path = Path(image)
    spec = mazeSpecification(image)
    fname = tempfile.NamedTemporaryFile().name
    f = open(fname, "w")
    f.write(spec)
    f.close()
    return { "args": [voxlogica_exe,fname], "label": path.name }

modelChecker_df = mk_df([ run(modelChecker(image)) for image in images ],"modelCheckingFull")


# Model Checking on the minimal graph using GraphLogicA

In [6]:
def graphSpecification():
    gql = f'''
    load graph = "mazeMin.json"
    let w = ap("cFFFFFF")
    let b = ap("c0000FF")
    let g = ap("c00FF00")
    let form1 = through(N b,w) & through(N g,w)
    let form2 = through(N (w & !through(N g,w)),b) & through(N g,w)
    let form3 = through(N form1,b)
    save "output/form1.json" form1
    save "output/form2.json" form2
    save "output/form3.json" form3
    '''
    fname = tempfile.NamedTemporaryFile().name
    f = open(fname,"w")
    f.write(gql)
    f.close()
    return run({"args" : [graphlogica_exe,fname], "label": "modelCheckingMin"},print_output=True)

graph_delta = graphSpecification()["delta"]



[        11ms] [info] GraphLogicA 0.4.1
[       208ms] [info] Parsing input...
[       243ms] [info] Preparing computation...
[       251ms] [info] Importing file "/home/vincenzo/data/local/repos/GraphLogicA/src/bin/release/net6.0/linux-x64/stdlib.imgql"
[       316ms] [info] Starting computation...
[       316ms] [info] Running 19 tasks
save to 'save to 'save to '"/home/vincenzo/data/local/repos/GraphLogicA/src/test/min/output/form1.json"': "/home/vincenzo/data/local/repos/GraphLogicA/src/test/min/output/form2.json"': "/home/vincenzo/data/local/repos/GraphLogicA/src/test/min/output/form3.json"': [|false; false; false; false; false; false; false|][|false; false; false; false; false; false; true|][|false; false; false; false; false; true; false|]


[       343ms] [info] ... done.



In [7]:
graph_true_delta = 0.02 # gotten by hand looking at the output

# Gather image sizes and produce the final table

In [8]:
def size(imgpath):    
    path = Path(imgpath)
    img = Image.open(imgpath)
    return { "pixels": img.width * img.height, "label": path.name}

size_df = pd.DataFrame([ size(image) for image in images]).set_index("label")

In [9]:
df = size_df.join(fakeConverter_df).join(converter_df).join(minimizer_df).join(modelChecker_df)
df["modelCheckingMin"] = graph_delta
df["modelCheckingMinLimit"] = graph_true_delta
df["idealTime"] = df["conversion"] + df["modelCheckingMinLimit"]
df["idealSpeedup"] = df["modelCheckingFull"] / df["idealTime"]
df.sort_values(by='pixels')

,pixels,conversion,conversionAndWrite,minimization,modelCheckingFull,modelCheckingMin,modelCheckingMinLimit,idealTime,idealSpeedup
label,,,,,,,,,
maze-128.png,16384,0.313659,0.448184,0.040439,0.407990,0.404271,0.02,0.333659,1.222775
maze-256.png,65536,0.309068,0.370617,0.124383,0.437770,0.404271,0.02,0.329068,1.330333
maze-512.png,262144,0.367882,0.642977,1.180128,0.477502,0.404271,0.02,0.387882,1.231051
maze-1024.png,1048576,0.350777,1.232414,1.941747,0.593413,0.404271,0.02,0.370777,1.600459
rai.png,2073600,0.407311,2.316466,3.923815,0.618447,0.404271,0.02,0.427311,1.447298
maze-2048.png,4194304,0.510984,4.181765,7.958157,0.902640,0.404271,0.02,0.530984,1.699937
maze-4096.png,16777216,1.085525,16.233358,32.402689,2.005657,0.404271,0.02,1.105525,1.814212
